In [ ]:
%pip install duckdb pandas numpy pyspainmobility requests

In [1]:
import duckdb
import pandas

BASE_PATH = '../../raw'
LAKE_LAYER = 'silver'

con = duckdb.connect('../../mobility.db')


def SQL(q):
    """Run SQL (printed for clarity) and return a DataFrame."""
    return con.execute(q).fetchdf()


print("DuckDB version:", con.sql(
    "SELECT version();").fetchone()[0])  # type: ignore

DuckDB version: v1.4.2


```sql
-- viajes distristos
-- se castea en vez de VARCHAR de 'si/no' a un booleano de true o false
CREATE TABLE silver_mitma_od_districts (
    fecha DATE, -- de TEXT a DATE
    hora SMALLINT,
    origen TEXT,
    destino TEXT,
    distancia TEXT,
    actividad_origen TEXT,
    actividad_destino TEXT,
    residencia TEXT,
    renta TEXT,
    edad TEXT,
    sexo TEXT,
    viajes DOUBLE,
    viajes_km DOUBLE,
    estudio_destino_posible BOOLEAN,  -- de VARCHAR a BOOLEAN
    estudio_origen_posible BOOLEAN,   -- de VARCHAR a BOOLEAN
);

-- Viajes municipios
-- silver_mitma_od_municipalities / bronze_mitma_viajes_municipios
CREATE TABLE silver_mitma_od_municipalities (
    fecha DATE,  -- de TEXT a DATE
    hora SMALLINT,
    origen TEXT,
    destino TEXT,
    distancia TEXT,
    actividad_origen TEXT,
    actividad_destino TEXT,
    residencia TEXT,
    renta TEXT,
    edad TEXT,
    sexo TEXT,
    viajes DOUBLE,
    viajes_km DOUBLE,
    estudio_destino_posible BOOLEAN, -- de VARCHAR a BOOLEAN
    estudio_origen_posible BOOLEAN,  -- de VARCHAR a BOOLEAN
);

-- Viajes GAU
-- bronze_mitma_od_gau / bronze_mitma_viajes_gau
CREATE TABLE silver_mitma_od_gau (
    fecha DATE,  -- de TEXT a DATE
    hora SMALLINT,
    origen TEXT,
    destino TEXT,
    distancia TEXT,
    actividad_origen TEXT,
    actividad_destino TEXT,
    residencia TEXT,
    renta TEXT,
    edad TEXT,
    sexo TEXT,
    viajes DOUBLE,
    viajes_km DOUBLE,
    estudio_destino_posible BOOLEAN, -- de VARCHAR a BOOLEAN
    estudio_origen_posible BOOLEAN,  -- de VARCHAR a BOOLEAN
);
```

In [2]:
def load_od_matrices(type="districts"):
    """ Load MITMA OD matrices from bronze to silver layer, transforming data types as needed."""

    dataset = 'od'
    table_name = f"{LAKE_LAYER}_mitma_{dataset}_{type}"

    SQL(f"DROP TABLE IF EXISTS {table_name};")
    SQL(f"""
        CREATE TABLE IF NOT EXISTS {table_name}(
            fecha DATE, -- de TEXT a DATE
            hora SMALLINT, -- Rename from 'periodo' to 'hora'. Converted to SMALLINT
            origen TEXT,
            destino TEXT,
            distancia TEXT,
            actividad_origen TEXT,
            actividad_destino TEXT,
            residencia TEXT,
            renta TEXT,
            edad TEXT,
            sexo TEXT,
            viajes DOUBLE, -- de VARCHAR a DOUBLE
            viajes_km DOUBLE, -- de VARCHAR a DOUBLE
            estudio_destino_posible BOOLEAN,  -- de VARCHAR a BOOLEAN
            estudio_origen_posible BOOLEAN,   -- de VARCHAR a BOOLEAN
        );
    """)

    SQL(f"""
        INSERT INTO {table_name}
        SELECT
            strptime(CAST(fecha AS VARCHAR), '%Y%m%d')::DATE AS fecha,
            CAST(periodo AS SMALLINT) AS hora,
            origen,
            destino,
            distancia,
            actividad_origen,
            actividad_destino,
            residencia,
            renta,
            edad,
            sexo,
            CAST(viajes AS DOUBLE) AS viajes,
            CAST(viajes_km AS DOUBLE) AS viajes_km,
            CASE 
                WHEN estudio_destino_posible IN ('si', 'SI', 'sí') THEN TRUE 
                WHEN estudio_destino_posible IN ('no', 'NO') THEN FALSE 
                ELSE NULL
            END AS estudio_destino_posible,
            CASE 
                WHEN estudio_origen_posible IN ('si', 'SI', 'sí') THEN TRUE 
                WHEN estudio_origen_posible IN ('no', 'NO') THEN FALSE 
                ELSE NULL
            END AS estudio_origen_posible,
        FROM bronze_mitma_{dataset}_{type};
""")

In [3]:
load_od_matrices(type="distritos")
load_od_matrices(type="municipios")
load_od_matrices(type="gau")

In [4]:
SQL(f"SELECT * FROM {LAKE_LAYER}_mitma_od_distritos LIMIT 5;")

,fecha,hora,origen,destino,distancia,actividad_origen,actividad_destino,residencia,renta,edad,sexo,viajes,viajes_km,estudio_destino_posible,estudio_origen_posible
0,2022-03-01,8,01001,01002,10-50,casa,frecuente,01,10-15,NA,NA,2.764,125.486,False,False
1,2022-03-01,17,01001,01002,10-50,no_frecuente,casa,01,10-15,NA,NA,6.526,303.751,False,False
2,2022-03-01,0,01001,01009_AM,2-10,casa,frecuente,01,10-15,NA,NA,10.279,67.832,False,False
3,2022-03-01,0,01001,01009_AM,2-10,frecuente,casa,01,>15,NA,NA,4.591,42.419,False,False
4,2022-03-01,2,01001,01009_AM,2-10,casa,frecuente,01,10-15,NA,NA,2.539,13.819,False,False


In [5]:
SQL(f"""
    SELECT '{LAKE_LAYER}_mitma_od_distritos' as name, count(*) FROM {LAKE_LAYER}_mitma_od_distritos
    UNION
    SELECT '{LAKE_LAYER}_mitma_od_municipios' as name, count(*) FROM {LAKE_LAYER}_mitma_od_municipios
    UNION
    SELECT '{LAKE_LAYER}_mitma_od_gau' as name, count(*) FROM {LAKE_LAYER}_mitma_od_gau;
""")

,name,count_star()
0,silver_mitma_od_municipios,34684127
1,silver_mitma_od_distritos,55862966
2,silver_mitma_od_gau,20827473


```sql
-- Distritos
CREATE TABLE IF NOT EXISTS bronze_mitma_people_day_districts (
  fecha DATE,
  zona_pernoctacion TEXT,
  edad TEXT,
  sexo TEXT,
  numero_viajes TEXT,   -- 0,1,2,2+ (mantener TEXT)
  personas DOUBLE,
  loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  source_file TEXT
);
```
```sql
-- Municipios
CREATE TABLE IF NOT EXISTS bronze_mitma_people_day_municipalities (
  fecha DATE,
  zona_pernoctacion TEXT,
  edad TEXT,
  sexo TEXT,
  numero_viajes TEXT,
  personas DOUBLE,
  loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  source_file TEXT
);
```
```sql
-- GAU
CREATE TABLE IF NOT EXISTS bronze_mitma_people_day_gau (
  fecha DATE,
  zona_pernoctacion TEXT,
  edad TEXT,
  sexo TEXT,
  numero_viajes TEXT,
  personas DOUBLE,
  loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  source_file TEXT
);
```

In [6]:
def load_people_day(type="districts", start_date='2022-03-01', end_date='2022-03-03'):
    """ Load MITMA People Day data from bronze to silver layer, transforming data types as needed. """
    dataset = 'people_day'
    table_name = f'{LAKE_LAYER}_mitma_{dataset}_{type}'

    SQL(f"DROP TABLE IF EXISTS {table_name};")
    SQL(f"""
        CREATE TABLE IF NOT EXISTS {table_name}(
            fecha DATE,
            zona_pernoctacion TEXT,
            edad TEXT,
            sexo TEXT,
            numero_viajes TEXT,
            personas DOUBLE,
        );
    """)

    SQL(f"""
        INSERT INTO {table_name}
        SELECT
            strptime(CAST(fecha AS VARCHAR), '%Y%m%d')::DATE AS fecha,
            zona_pernoctacion,
            edad,
            sexo,
            numero_viajes,
            CAST(personas AS DOUBLE) AS personas,
        FROM bronze_mitma_{dataset}_{type};
    """)

In [7]:
load_people_day(type="distritos")
load_people_day(type="municipios")
load_people_day(type="gau")

In [8]:
SQL(f"SELECT * FROM {LAKE_LAYER}_mitma_people_day_distritos LIMIT 5;")

,fecha,zona_pernoctacion,edad,sexo,numero_viajes,personas
0,2022-03-01,01001,0-25,hombre,0,125.296
1,2022-03-01,01001,0-25,hombre,2,115.378
2,2022-03-01,01001,0-25,hombre,2+,176.630
3,2022-03-01,01001,0-25,mujer,0,125.069
4,2022-03-01,01001,0-25,mujer,2,117.712


In [9]:
SQL(f"""
    SELECT '{LAKE_LAYER}_mitma_people_day_distritos' as name, count(*) FROM {LAKE_LAYER}_mitma_people_day_distritos
    UNION
    SELECT '{LAKE_LAYER}_mitma_people_day_municipios' as name, count(*) FROM {LAKE_LAYER}_mitma_people_day_municipios
    UNION
    SELECT '{LAKE_LAYER}_mitma_people_day_gau' as name, count(*) FROM {LAKE_LAYER}_mitma_people_day_gau;
""")

,name,count_star()
0,silver_mitma_people_day_gau,182214
1,silver_mitma_people_day_distritos,340448
2,silver_mitma_people_day_municipios,231688


```sql
-- Distritos
CREATE TABLE IF NOT EXISTS bronze_mitma_overnight_stay_districts (
  fecha DATE,
  zona_residencia TEXT,
  zona_pernoctacion TEXT,
  personas DOUBLE,
  loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  source_file TEXT
);
```
```sql
-- Municipios
CREATE TABLE IF NOT EXISTS bronze_mitma_overnight_stay_municipalities (
  fecha DATE,
  zona_residencia TEXT,
  zona_pernoctacion TEXT,
  personas DOUBLE,
  loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  source_file TEXT
);
```
```sql
-- GAU
CREATE TABLE IF NOT EXISTS bronze_mitma_overnight_stay_gau (
  fecha DATE,
  zona_residencia TEXT,
  zona_pernoctacion TEXT,
  personas DOUBLE,
  loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  source_file TEXT
);
```

In [10]:

def load_overnight_stay(type="districts"):
    dataset = 'overnight_stay'
    table_name = f'{LAKE_LAYER}_mitma_{dataset}_{type}'

    SQL(f"DROP TABLE IF EXISTS {table_name};")
    SQL(f"""
        CREATE TABLE IF NOT EXISTS {table_name}(
            fecha DATE,
            zona_residencia TEXT,
            zona_pernoctacion TEXT,
            personas DOUBLE,
        );
    """)

    SQL(f"""
        INSERT INTO {table_name}
        SELECT
            strptime(CAST(fecha AS VARCHAR), '%Y%m%d')::DATE AS fecha,
            zona_residencia,
            zona_pernoctacion,
            CAST(personas AS DOUBLE) AS personas,
        FROM bronze_mitma_{dataset}_{type};
""")

In [11]:
load_overnight_stay(type="distritos")
load_overnight_stay(type="municipios")
load_overnight_stay(type="gau")

In [12]:
SQL(f"SELECT * FROM {LAKE_LAYER}_mitma_overnight_stay_distritos LIMIT 5;")

,fecha,zona_residencia,zona_pernoctacion,personas
0,2022-03-01,01001,01001,2733.784
1,2022-03-01,01001,01004_AM,2.514
2,2022-03-01,01001,01009_AM,18.431
3,2022-03-01,01001,01017_AM,2.922
4,2022-03-01,01001,01051,7.831


In [13]:
SQL(f"""
    SELECT '{LAKE_LAYER}_mitma_overnight_stay_distritos' as name, count(*) FROM {LAKE_LAYER}_mitma_overnight_stay_distritos
    UNION
    SELECT '{LAKE_LAYER}_mitma_overnight_stay_municipios' as name, count(*) FROM {LAKE_LAYER}_mitma_overnight_stay_municipios
    UNION
    SELECT '{LAKE_LAYER}_mitma_overnight_stay_gau' as name, count(*) FROM {LAKE_LAYER}_mitma_overnight_stay_gau;
""")

,name,count_star()
0,silver_mitma_overnight_stay_distritos,947839
1,silver_mitma_overnight_stay_municipios,741266
2,silver_mitma_overnight_stay_gau,574703


```sql
-- Distritos
CREATE TABLE IF NOT EXISTS bronze_mitma_districts (
  id TEXT,
  name TEXT,
  population DOUBLE,
  geometry GEOMETRY,
  municipio_id TEXT,
);
```
```sql
-- Municipios
CREATE TABLE IF NOT EXISTS bronze_mitma_municipalities (
  id TEXT,
  name TEXT,
  population DOUBLE,
  geometry GEOMETRY,
);
```
```sql
-- GAU
CREATE TABLE IF NOT EXISTS bronze_mitma_gau (
  id TEXT,
  name TEXT,
  population DOUBLE,
  geometry GEOMETRY,
);
```

In [ ]:

def load_zones(type="districts", id_length=None):
    table_name = f'{LAKE_LAYER}_mitma_{type}'

    SQL("""
        INSTALL spatial;
        LOAD spatial;
    """)

    SQL(f"DROP TABLE IF EXISTS {table_name};")
    SQL(f"""
        CREATE TABLE IF NOT EXISTS {table_name}(
            id TEXT,
            name TEXT,
            population DOUBLE,
            geometry GEOMETRY,
        );
    """)

    where_clause = f"WHERE length(id) = {id_length}" if id_length is not None else ""

    SQL(f"""
        INSERT INTO {table_name}
        SELECT
            id,
            name,
            CAST(population AS DOUBLE) AS population,
            ST_Multi(ST_GeomFromText(geometry)) AS geometry
        FROM bronze_mitma_{type}
        {where_clause}
        ORDER BY id ASC;
    """)

In [16]:
load_zones(type="distritos", id_length=7)
load_zones(type="municipios", id_length=5)
load_zones(type="gau", id_length=8)

In [ ]:
# metemos la columna municipio_id en la tabla de distritos.
# pues el id de un distrito es el de su municipio más un sufijo de dos dígitos.
# 28079 (Municipio de Madrid) -> 2807901 (Distrito de Centro)

type = "distritos"

SQL(f"""
    ALTER TABLE {LAKE_LAYER}_mitma_{type} ADD COLUMN municipio_id TEXT;
""")

SQL(f"""
    UPDATE {LAKE_LAYER}_mitma_{type}
    SET municipio_id = CASE
        WHEN length(id) = 7 THEN substring(id, 1, 5)
        WHEN length(id) = 5 THEN id
        ELSE NULL
    END;
""")

In [62]:
SQL(f"""
    SELECT 
        * EXCLUDE (geometry), 
        ST_AsGeoJSON(geometry) AS geojson 
    FROM {LAKE_LAYER}_mitma_distritos
    WHERE municipio_id = '01059'
    LIMIT 10;
""")

,id,name,population,municipio_id,geojson
0,0105901,Vitoria-Gasteiz distrito 01,38600.0,01059,"{""type"":""MultiPolygon"",""coordinates"":[[[[-2.66..."
1,0105902,Vitoria-Gasteiz distrito 02,69807.0,01059,"{""type"":""MultiPolygon"",""coordinates"":[[[[-2.67..."
2,0105903,Vitoria-Gasteiz distrito 03,54114.0,01059,"{""type"":""MultiPolygon"",""coordinates"":[[[[-2.66..."
3,0105904,Vitoria-Gasteiz distrito 04,33042.0,01059,"{""type"":""MultiPolygon"",""coordinates"":[[[[-2.66..."
4,0105905,Vitoria-Gasteiz distrito 05,52432.0,01059,"{""type"":""MultiPolygon"",""coordinates"":[[[[-2.69..."
5,0105906,Vitoria-Gasteiz distrito 06,5098.0,01059,"{""type"":""MultiPolygon"",""coordinates"":[[[[-2.64..."


In [60]:
SQL(f"""
    SELECT 
        * EXCLUDE (geometry), 
        ST_AsGeoJSON(geometry) AS geojson 
    FROM {LAKE_LAYER}_mitma_municipios
    WHERE id = '01059'
    LIMIT 5;
""")

,id,name,population,geojson
0,01059,Vitoria-Gasteiz,253093.0,"{""type"":""MultiPolygon"",""coordinates"":[[[[-2.64..."


In [57]:
SQL(f"""
    SELECT '{LAKE_LAYER}_mitma_distritos' as name, count(*) FROM {LAKE_LAYER}_mitma_distritos
    UNION
    SELECT '{LAKE_LAYER}_mitma_municipios' as name, count(*) FROM {LAKE_LAYER}_mitma_municipios
    UNION
    SELECT '{LAKE_LAYER}_mitma_gau' as name, count(*) FROM {LAKE_LAYER}_mitma_gau
    ORDER BY count(*) DESC;
""")

,name,count_star()
0,silver_mitma_municipios,1645
1,silver_mitma_distritos,1565
2,silver_mitma_gau,911


```sql
-- Distritos
CREATE TABLE IF NOT EXISTS bronze_ine_padron_municipios (
  cod        VARCHAR,
  nombre     VARCHAR,
  fk_unidad  INTEGER,
  fk_escala  INTEGER,
  data_txt   TEXT,
  data       JSON,
  -- Columnas extras añadidas para auditoria. 
  loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  -- loaded_by TEXT DEFAULT CURRENT_USER,
  source_file TEXT
);

In [63]:
con.close()